# I. Import Needed Libraries

Selenium: accesses and controls web browser to submit search and locate web elements by xpath

CSV: reads and writes CSV files

re: regular expression for pattern matching

requests: makes HTTP requests

BeautifulSoup:parses HTML

pandas: data analysis library to manage dataframes


In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import csv, re, requests
from bs4 import BeautifulSoup
import pandas as pd

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


# II. Instantiate Browser Object 

In [2]:
url = 'https://ww2.amstat.org/meetings/jsm/2011/onlineprogram/'

# chromedriver needs to be in exectuable path or you need to map to where your chromedriver is
browser = webdriver.Chrome()
browser.get(url)


# III. Navigate through webpage and scrape objects

### Creat xpaths for needed items

In [3]:
xpath_search = '//*[@id="content_full"]/table[1]/tbody/tr/td/form/input[@type = "submit"]'
xpath_session = '//tr//td[1]//a[contains(@href, "ActivityDetails")]'
xpath_titles = '//tbody//tr//td[2]//a[contains(@href, "ActivityDetails")]'
xpath_sponsors = '//tbody//tr//td[3]'
xpath_type = '//tbody//tr/td[4]'

### Click search with empty fields

In [5]:
# wouldn't click on this page so had to execute script instead of standard click() method
element = browser.find_element_by_xpath(xpath_search)
browser.execute_script("arguments[0].click();", element)


# browser.find_element_by_xpath(xpath_search).click()
# wait = WebDriverWait(browser, 30)
# wait.until(EC.presence_of_element_located((By.XPATH, xpath_search)))
# browser.find_element_by_xpath(xpath_search).click()

### Create lists of each field needed

In [6]:
sessions = [item.text for item in browser.find_elements_by_xpath(xpath_session) if 'Session' not in item.text]
titles = [item.text for item in browser.find_elements_by_xpath(xpath_titles)]
sponsors = [item.text.replace('\n', ', ') for item in browser.find_elements_by_xpath(xpath_sponsors) 
            if item.text != 'Sponsor(s)']
session_type = [item.text for item in browser.find_elements_by_xpath(xpath_type) 
                if item.text != "Type" ]
session_linkToAbstract = [item.get_attribute('href') for item in browser.find_elements_by_xpath(xpath_titles)]

In [7]:
len(sessions) == len(titles) == len(sponsors) == len(session_linkToAbstract)

True

In [8]:
df = pd.DataFrame({'Session_No' : sessions,
                   'Session_Title' : titles,
                   'Sponsors': sponsors,
                   'Type': session_type,
                   'Session_URL': session_linkToAbstract                   
                  })

In [9]:
df['Session_ID'] = df['Session_URL'].apply(lambda x: re.sub(r'https\S+((\d){6})', r'\1', x))

In [10]:
df.head(1)

,Session_No,Session_Title,Session_URL,Sponsors,Type,Session_ID
0,207119,CANCELLED: ASA Development Committee Meeting,https://ww2.amstat.org/meetings/jsm/2011/onlin...,Development Committee,Other Cmte/Business,207119


In [11]:
df.drop_duplicates(subset = 'Session_URL', inplace = True)
df.reset_index(inplace = True, drop= True)

In [12]:
dest = r'C:\Users\Thaunga\Scripts\02. Presentations\JSM\Datasets'
df.to_excel(r'{}\2011Sessions.xlsx'.format(dest))

# Test Code for this year's object locations (xpaths)

In [13]:
df[df['Session_No'].apply(lambda x: bool(re.search(r'(?<!\d)\d{1,3}(?!\d)',x)))]

,Session_No,Session_Title,Session_URL,Sponsors,Type,Session_ID
8,CE_01C,Foundations and Recent Advances in Longitudina...,https://ww2.amstat.org/meetings/jsm/2011/onlin...,ASA,Continuing Education Course,206444
9,CE_04C,Administrative Data for Statistical Purposes,https://ww2.amstat.org/meetings/jsm/2011/onlin...,ASA,Continuing Education Course,206415
10,CE_05C,Wavelets in the Real World,https://ww2.amstat.org/meetings/jsm/2011/onlin...,"ASA, Section on Nonparametric Statistics",Continuing Education Course,206416
30,CE_06C,Data Stream Mining: Tools and Applications,https://ww2.amstat.org/meetings/jsm/2011/onlin...,"ASA, Section on Statistical Computing",Continuing Education Course,206413
31,CE_01C,Foundations and Recent Advances in Longitudina...,https://ww2.amstat.org/meetings/jsm/2011/onlin...,ASA,Continuing Education Course,206845
32,CE_07C,Bayesian Inference,https://ww2.amstat.org/meetings/jsm/2011/onlin...,"ASA, Section on Bayesian Statistical Science",Continuing Education Course,206427
33,CE_08C,Advanced Topics in Survey Sampling,https://ww2.amstat.org/meetings/jsm/2011/onlin...,"ASA, Section on Survey Research Methods",Continuing Education Course,206442
34,CE_09C,Fundamental Statistics Concepts in Presenting ...,https://ww2.amstat.org/meetings/jsm/2011/onlin...,"ASA, Section on Statistical Graphics",Continuing Education Course,206431
51,CE_10C,Risk Analysis in National Security,https://ww2.amstat.org/meetings/jsm/2011/onlin...,"ASA, Section on Statistics in Defense and Nati...",Continuing Education Course,206392
52,9,Including Disease Models and Biomarkers in the...,https://ww2.amstat.org/meetings/jsm/2011/onlin...,"Biopharmaceutical Section, International India...",Invited Papers,206379


In [23]:
test = df['Session_URL'].iloc[52]
test

'https://ww2.amstat.org/meetings/jsm/2011/onlineprogram/ActivityDetails.cfm?SessionID=206379'

In [24]:
r = requests.get(test)
soup = BeautifulSoup(r.text, 'lxml')

In [28]:
[soup.find_all('tr')[1].find('b').text]

[' 9 !']

# Run Code to extract workshops and abstract_urls

In [29]:
# dataframe of abstracts to scrape with new urls
session_url = []
abstract_url = []
abstract_title = []
session_no_v2 = []

#dataframe of abstracts already written in the session
workshop_abstract = []
workshop_session_no = []
workshop_session_url = []

for link in session_linkToAbstract:
    r = requests.get(link)
    soup = BeautifulSoup(r.text, 'lxml')
    
    # extract abstract titles and links
    abstract_links = [item.get('href') for item in soup.find_all('a') if re.search('AbstractDetails', str(item.get('href')))]
    
    if len(abstract_links) == 0:
        try:
            abstract_workshop = re.sub(r'(\n|\r|\t)', '', soup.find_all('tr')[5].text)
            id_no = soup.find_all('tr')[1].find('b').text.strip()
            
            workshop_abstract.append(abstract_workshop)
            workshop_session_no.append(id_no)
            workshop_session_url.append(link)
        except:
            continue
    
    else:
        abstract_titles = [item.text for item in soup.find_all('a') 
            if re.search('AbstractDetails', str(item.get('href')))]

        # map back to session id and original link for later joining
        session_ids = [soup.find_all('tr')[1].find('b').text] * len(abstract_links)
        original_link = [link] * len(abstract_links)

        # create list of 
        session_url = session_url + original_link
        abstract_url = abstract_url + abstract_links
        abstract_title = abstract_title + abstract_titles
        session_no_v2 = session_no_v2 + session_ids

    

In [30]:
len(session_url) == len(abstract_url) == len(abstract_title) == len(session_no_v2)

True

In [31]:
len(session_url)

5230

In [32]:
len(workshop_abstract) == len(workshop_session_no) == len(workshop_session_url)

True

In [33]:
len(workshop_abstract)

35

In [34]:
df_abstracts = pd.DataFrame({'Session_URL': session_url,
                            'Abstract_URL' : abstract_url,
                            'Abstract_Title': abstract_title,
                            'Session_No_v2': session_no_v2})

In [35]:
df_abstracts['Session_ID'] = df_abstracts['Session_URL'].apply(lambda x: re.sub(r'https\S+((\d){6})', r'\1', x))
df_abstracts['Abstract_URL'] = df_abstracts['Abstract_URL'].apply(lambda x: url + x)

In [36]:
df_abstracts.head(1)

,Abstract_Title,Abstract_URL,Session_No_v2,Session_URL,Session_ID
0,The Use of Drug Disease Models and Biomarkers ...,https://ww2.amstat.org/meetings/jsm/2011/onlin...,9 !,https://ww2.amstat.org/meetings/jsm/2011/onlin...,206379


In [37]:
df_workshops = pd.DataFrame({'Session_No': workshop_session_no,
                            'Abstract_Text': workshop_abstract,
                            'Session_URL': workshop_session_url})


In [38]:
df_workshops['Session_ID']= df_workshops['Session_URL'].apply(lambda x:re.sub(r'https\S+((\d){6})', r'\1', x))
df_workshops = df_workshops[df_workshops['Abstract_Text'].str.contains('Organizer\(s\)|Chair\(s\)') == False]

In [39]:
df_workshops.head(3)

,Abstract_Text,Session_No,Session_URL,Session_ID
0,We first present linear mixed models for conti...,CE_01C,https://ww2.amstat.org/meetings/jsm/2011/onlin...,206444
1,There is a growing interest in developing regi...,CE_04C,https://ww2.amstat.org/meetings/jsm/2011/onlin...,206415
2,Statistical modeling with wavelets has become ...,CE_05C,https://ww2.amstat.org/meetings/jsm/2011/onlin...,206416


In [40]:
df_abstracts.drop_duplicates(subset = 'Abstract_URL', inplace = True)
df_abstracts.reset_index(drop = True, inplace = True)

In [41]:
df_abstracts.to_excel(r'{}\2011Abstracts.xlsx'.format(dest))
df_workshops.to_excel(r'{}\2011Workshops.xlsx'.format(dest))

# Test extraction of abstract info here

In [42]:
test = df_abstracts['Abstract_URL'].iloc[0]
test

'https://ww2.amstat.org/meetings/jsm/2011/onlineprogram/AbstractDetails.cfm?abstractid=300369'

In [43]:
r = requests.get(test)
soup = BeautifulSoup(r.text, 'lxml')

In [47]:
re.sub(r'\r|\n|\t', '', soup.find_all('tr')[11].find('p').text)

"Drug disease models can be leveraged to improve trial design. For example, virtual patient responses can be simulated from drug disease models and used to assess a design's performance. If a trial performs poorly under this scenario of truth, the design needs to be modified. Knowledge of drug disease models can also be leveraged in the selection of final data analysis models and the definition of prior distributions if a Bayesian analysis is employed. Adaptive designs require simulation to understand the operating characteristics of the design, and the use of both drug disease models and biomarkers can improve performance. In this presentation, several examples (fixed designs and adaptive designs) will be presented where drug disease models and biomarkers have been used to optimize design. "

# Code to extract abstract details from abstract URLs

In [48]:
abs_num = []
abs_keyword = []
abs_type = []
abs_text = []

for link in df_abstracts['Abstract_URL']:
    r = requests.get(link)
    soup = BeautifulSoup(r.text, 'lxml')
    
    # abstract number
    try:
        number = re.sub(r'\S+abstractid=(\d+)', r'\1', link)
        abs_num.append(number)
    except:
        abs_num.append(None)
    
    # abstract keywords
    try:
        keywords = [item.text for item in soup.find_all('a') if re.search('keyword', str(item.get('href')))]
        keywords = ', '.join(keywords)
        abs_keyword.append(keywords)
    except:
        abs_keyword.append(None)
        
    
    # abstract type
    try:
        talk_type = re.sub(r'Type:|\n|\r|\t', '', soup.find_all('tr')[2].text)
        abs_type.append(talk_type)
    except:
        abs_type.append(None)
    
    # abstract text
    try:
        abstract_text = re.sub(r'\r|\n|\t', '', soup.find_all('tr')[11].find('p').text)
        abs_text.append(abstract_text)
    except:
        abs_text.append(None)
    

In [49]:
df_abstracts['Abstract_ID'] = abs_num
df_abstracts['Abstract_Keywords'] = abs_keyword
df_abstracts['Abstract_Type'] = abs_type
df_abstracts['Abstract_Text'] = abs_text

In [50]:
df_abstracts.to_excel(r'{}\2011Abstracts.xlsx'.format(dest))

In [51]:
df_abstracts.head(1)

,Abstract_Title,Abstract_URL,Session_No_v2,Session_URL,Session_ID,Abstract_ID,Abstract_Keywords,Abstract_Type,Abstract_Text
0,The Use of Drug Disease Models and Biomarkers ...,https://ww2.amstat.org/meetings/jsm/2011/onlin...,9 !,https://ww2.amstat.org/meetings/jsm/2011/onlin...,206379,300369,"modeling, biomarkers, simulation, design, adap...",Invited,Drug disease models can be leveraged to improv...


In [52]:
df_details = df_workshops.append(df_abstracts)

In [53]:
df.head(0)

,Session_No,Session_Title,Session_URL,Sponsors,Type,Session_ID


In [54]:
df_final = df_details.merge(df[['Session_Title', 'Sponsors', 'Type', 'Session_ID']], on = 'Session_ID', how= 'outer')

In [55]:
df_final['Unique'] = df_final['Session_ID'].map(str) + df_final['Abstract_ID'].map(str)

In [56]:
df_final.drop_duplicates(subset= 'Unique', inplace = True)
df_final.reset_index(drop = True, inplace = True)

In [57]:
df_final['Session_No'] = df_final['Session_No'].apply(lambda x: x if bool(str(x).startswith('CE') or str(x)[0].isdigit()) else "")

In [58]:
df_final['Session_No_Keep'] = df_final['Session_No_v2'].combine_first(df_final['Session_No'])

In [59]:
del df_final['Session_No']
del df_final['Session_No_v2']

In [60]:
df_final.rename(columns = {'Session_No_Keep': 'Session_No'}, inplace=True)

In [61]:
new_dest = r'C:\Users\Thaunga\Scripts\02. Presentations\JSM\Final Data'
df_final.to_excel(r'{}\2011_JSM_data.xlsx'.format(new_dest))